In [4]:
print("ok")

ok


In [5]:
%pwd

'c:\\Users\\Chinthan BD\\End_To_End_Medical_Chatbot_GenerativeAI\\research'

In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\Chinthan BD\\End_To_End_Medical_Chatbot_GenerativeAI'

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def load_pdf_file(data):
  loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
  documents=loader.load()
  return documents

In [10]:
extracted_data = load_pdf_file(data='Data/')

In [11]:
def text_split(extracted_data):
  text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
  text_chunks=text_splitter.split_documents(extracted_data)
  return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks",len(text_chunks))

Length of Text Chunks 40000


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()


C:\Users\Chinthan BD\AppData\Local\Temp\ipykernel_12632\3967912968.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [15]:
import pinecone
print(pinecone.__version__)


3.0.0


In [16]:
from dotenv import load_dotenv
load_dotenv()
import os

In [17]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [18]:
import os
from pinecone import Pinecone, ServerlessSpec

api_key = os.getenv("PINECONE_API_KEY")
if not api_key:
    raise ValueError("PINECONE_API_KEY environment variable not set")

pc = Pinecone(api_key=api_key)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


In [19]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [20]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
  documents = text_chunks,
  index_name = index_name,
  embedding=embeddings,
)

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
  index_name = index_name,
  embedding=embeddings,
)

In [22]:
docsearch

In [23]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [24]:
retrieved_docs=retriever.invoke("what is Acne?")

In [25]:
retrieved_docs

[Document(id='b97ba4cc-d580-4955-a727-2879dd4b420c', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\medical_textbook.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='cb2aab17-c76f-4453-8165-75749f92dbb5', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\medical_textbook.pdf', 'total_pages': 4505.0}, page_content='Sebaceous follicles— A structure found within the\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— An oily skin moisturizer produced by\nsebaceous glands.\nTretinoin— A drug that works by increasing the\nt

In [38]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_API_KEY

In [47]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Load model locally
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512
)

llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cpu


In [48]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)


rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [49]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])


Sebaceous follicles— A structure found within the skin that houses the oil-producing glands and hair follicles, where pimples form. Sebum— An oily skin moisturizer produced by sebaceous glands. Tretinoin— A drug that works by increasing the turnover (death and replacement) of skin cells.
